In [1]:
import pandas as pd
import numpy as np

# Deck

## Requirement

- suits not required
- multiple decks
- need to be able to shuffle the deck
- need to draw a card from the deck

## Plan

- class: deck
    - can take number of decks as a variable
    - default 1 deck
- method: shuffle
- method: deal
- attribute: list of all the cards in the 'deck'

## Code

In [172]:
class Deck():
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values):
        # Multiply values by number of suits
        # Then by number of decks
        self.num_decks = num_decks
        self.deck = values * 4 * self.num_decks
        
    def __str__(self):
        return "{} decks, {} cards left".format(self.num_decks, len(self.deck))
    
    def shuffle(self):
        np.random.shuffle(self.deck)
    
    def deal(self, hand=None):
        if hand == None:
            return self.deck.pop(0)
        else:
            hand.append(self.deck.pop(0))

In [21]:
test = Deck()
print (test)
print (test.deck)
print (test.deal())
print (test.deck)

1 decks, 52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
A
[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']


# Calculate points

## Requirement

- Take a hand
- Calculate how many points there are in the hand
    - J, Q, K are 10
    - Ace could be 1 or 11
        - How do we decide?
        - If 11 doesn't bust, then 11, else 1
            - We have to calculate the rest of the cards first

## Plan

- Remove Aces in the hand
    - Track how many Aces there were
- Count J, Q, K as 10

## Code

In [22]:
def calculate_points(hand):
    points = 0
    num_ace = 0
    
    # Deal with aces
    while "A" in hand:
        hand.remove("A")
        num_ace += 1
    
    # Deal with the rest of the hand
    for i in hand:
        try:
            # If it's a number, add it to points
            points += i
        except:
            # If it's not a number, it's a 10
            points += 10
            
    # deal with the aces
    for i in range(num_ace):
        if points + 11 <= 21:
            points += 11
        else:
            points += 1
    return points

In [23]:
calculate_points([5, 5, "A"])

21

# Simulate game

## Requirements

- play one game
- return the data we need

In [156]:
data_dictionary = pd.read_csv("data_dictionary.csv")
data_dictionary

,Feature,Type,Description
0,num_decks,Int,Number of decks used
1,dealer_open,Int,The card we can see
2,dealer_initial,Int,The dealer's starting points
3,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
4,dealer_num_hits,Int,Number of times the dealer hit
5,dealer_final,Int,The dealer's final points
6,dealer_busts,Binary,"1 - dealer busts, 0 - dealer did not bust"
7,dealer_hand,Obj,"List of the dealer's cards, in the order that ..."
8,player_inital,Int,The player's starting points
9,player_hit,Binary,"1 - player hit, 0 - player did not hit"


## Outcomes

- player gets blackjack, dealer didn't
    - player wins
- dealer gets blackjack, player didn't
    - player loses
- player hits and busts
    - player loses
- player hits, dealer hits and busts
    - player wins
- player hits, dealer hits/ dealer >= hard 17
    - player > dealer
        - player wins
    - player < dealer
        - player loses
    - player == dealer
        - draw

## Plan

- create deck
- deal initial hand
- check if anyone has a blackjack
- no blackjacks, player goes first
    - in any case, if player has anything <= 11, hit
    - if anything 12 or higher
        - random
            - randomize whether to hit or not if smaller than 18
        - recommended
            - if dealer showing <= 6 and player <= 16, hit
- once player is done, dealer goes
    - if dealer has anything smaller than 17, he must hit
    - if dealer has a soft 17, he must hit

In [173]:
def sim_game(num_decks=1, strategy=0):
    game_deck = Deck(num_decks=num_decks)
    game_deck.shuffle()
    
    dealer_hand = []
    player_hand = []
    
    # deal 2 cards each
    for i in range(2):
        game_deck.deal(player_hand)
        game_deck.deal(dealer_hand)
        
    # get their initial points
    # copy so that our hand is not changed
    player_initial = calculate_points(player_hand.copy())
    dealer_initial = calculate_points(dealer_hand.copy())
    
    # get dealer open card
    dealer_open = dealer_hand[0]
    
    # Change it to numerics
    if dealer_open in ["J", "Q", "K"]:
        dealer_open = 10
    elif dealer_open == "A":
        dealer_open = 1
    
    dealer_hit = 0
    dealer_num_hits = 0
    player_hit = 0
    player_num_hits = 0
    player_busts = 0
    dealer_busts = 0
    dealer_final = calculate_points(dealer_hand.copy())
    player_final = calculate_points(player_hand.copy())

    
    # if anyone got a blackjack, the game should end
    if player_initial != 21 and dealer_initial != 21:
        # if neither of them got a blackjack game continues
        # the player goes first
        
        # If player <= 11, hit
        while calculate_points(player_hand.copy()) <= 11:
            player_hit += 1
            game_deck.deal(player_hand)
        
        # If strategy is random, randomize hit for 18 and below
        if strategy == 0:
            while calculate_points(player_hand.copy()) <= 18:
                if np.random.random() < 0.5:
                    player_hit += 1
                    game_deck.deal(player_hand)
        # If strategy is recommended, stand on 17 and above
        else:
            while calculate_points(player_hand.copy()) <= 16:
                if dealer_open <= 6:
                    player_hit += 1
                    game_deck.deal(player_hand)
        
        # update player's final
        player_final = calculate_points(player_hand.copy())
        
        # dealer's turn
        
        # If dealer < 17, hit
        while calculate_points(dealer_hand.copy()) < 17:
            dealer_hit += 1
            game_deck.deal(dealer_hand)
            
        # If dealer has a soft 17, hit
        if calculate_points(dealer_hand.copy()) == 17:
            if "A" in dealer_hand:
                dealer_copy = dealer_hand.copy()
                dealer_copy.remove("A")
                if calculate_points(dealer_copy.copy()) == 6:
                    dealer_hit += 1
                    game_deck.deal(dealer_hand)
        
        # update dealer's final
        dealer_final = calculate_points(dealer_hand.copy())
        
    player_loses = 0
    draw = 0
    player_wins = 0
    
    # Check who wins
    if player_initial == 21 and dealer_initial != 21:
        player_wins = 1
    elif dealer_initial == 21 and player_initial != 21:
        player_loses = 1
    elif player_busts:
        player_loses = 1
    elif dealer_busts:
        player_wins = 1
    elif player_final > dealer_final:
        player_wins = 1
    elif player_final < dealer_final:
        player_loses = 1
    elif player_final == dealer_final:
        draw = 1
    
    return [num_decks, dealer_open, dealer_initial, dealer_hit, dealer_num_hits, dealer_final, dealer_busts, dealer_hand, \
            player_initial, player_hit, player_num_hits, player_final, player_busts, player_hand, \
            player_loses, draw, player_wins, strategy]

In [174]:
print (sim_game())

[1, 10, 14, 2, 0, 20, 0, [10, 4, 2, 4], 14, 3, 0, 20, 0, ['A', 3, 9, 'A', 6], 0, 1, 0, 0]


# Generate a dataframe

## Requirements

- run sim_game() for some number of times and save the output to a df
- maybe also track the number of decks we use?

In [175]:
def gen_df(num_decks=1, df_size=5000):
    data = []
    for i in range(df_size//2):
        data.append(sim_game(num_decks=num_decks, strategy=0))
    for i in range(df_size//2):
        data.append(sim_game(num_decks=num_decks, strategy=1))
                   
    return pd.DataFrame(data, columns=data_dictionary.Feature.values)

In [ ]:
df = gen_df(num_decks=4)

# Testing our code

In [ ]:
# When player_loses = 1, draw = 0, player_wins = 0
# When player_loses = 0, draw = 1, player_wins = 0
# When player_loses = 0, draw = 0, player_wins = 1
# When dealer_busts = 1, player_loses = 0, draw = 0, player_wins = 1
# When player_busts = 1, player_loses = 1, draw = 0, player_wins = 0, dealer_hit = 0, dealer_busts = 0
# When dealer_busts = 0 and player_busts = 0, if player_final > dealer final, player_loses = 0, draw = 0, player_wins = 1
# When dealer_busts = 0 and player_busts = 0, if player_final = dealer final, player_loses = 0, draw = 1, player_wins = 0
# When dealer_busts = 0 and player_busts = 0, if player_final < dealer final, player_loses = 1, draw = 0, player_wins = 0